In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: John, Gen, Tim
### Team Strategy Chosen: Risky 

Requirements for this assignment:
- Code needs to be dynamic
- Must read in a csv file containing a finite number of stock tickers (with an example csv file in this repo)
- Portfolio must contain only US listed stocks
- Must have an average daily volume of at least 10 000 shares as calculated based on the time interval July 2, 2021 to October 22, 2021
- Pick a minimum 10 maximum 20 stocks for portfolio.
- If we choose n stocks, each stock must make up minimum (100/(2n))% of the portfolio when weighted by value as of closing prices on November 26, 2021
- No individual stock may make up more than 35% of the portfolio when weighted by value
- We have $100000 USD to spend on portfolio and all must be spent
- When code is run with the .csv file, it must create a DataFrame "FinalPortfolio"
    - With this portfolio, index starts at 1 and ends at the number of stocks that our code chooses.
    - Headings must be: Ticker, Price, Shares, Value, Weight
        - Ticker is the ticker selected
        - Price is the price on November 26, 2021
        - Shares is the number of shares purchased (can be fractional)
        - Value is the total value of those shares
        - Weighted is the weight that the value of shares represents relative to the value of the portfolio (which is $100,000)
    - Needs to show that the total adds up to $100,000
    - Also need to show the weights add to 100%
    - This DataFrame must be printed to the screen as the second to last output to the screen.
- After the DataFrame, one final DataFrame called "Stocks" which has the same index as "Final Portfolio" must be mad
    - Only has the Tickers and Shares from "Final Portfolio"
    - Must output this DataFrame to a csv file titled "Stocks_Group_XX.csv"

### Step #1: Filter out valid US tickers

In [3]:
stocks_from_csv = pd.read_csv('Tickers.csv')
stocks_from_csv


,AAPL
0,ABBV
1,ABT
2,ACN
3,AGN
4,AIG
...,...
56,TXN
57,UNH
58,UNP
59,UPS


In [4]:
stocks_lst = stocks_from_csv.iloc[:,0]
stocks_lst

0     ABBV
1      ABT
2      ACN
3      AGN
4      AIG
      ... 
56     TXN
57     UNH
58     UNP
59     UPS
60     USB
Name: AAPL, Length: 61, dtype: object

In [5]:
stocks_from_csv.columns[0]

'AAPL'

In [113]:
current_stock=yf.Ticker(stocks_lst[0])
current_stock.info['market']

'us_market'

In [122]:
#try doing the process below with threading
from concurrent.futures import ThreadPoolExecutor, as_completed

us_stock_lst = []
stocks_lst
 
def download_file(ticker_name):
    try:
        ticker_yf = yf.Ticker(ticker_name) # Store as variable to optimize access time

        if (ticker_yf.info['regularMarketPrice'] != None) and (ticker_yf.info['market'] == 'us_market'):
            us_stock_lst.append(ticker_yf)
    except requests.exceptions.RequestException as e:
       return e
 
def runner():
    threads= []
    with ThreadPoolExecutor(max_workers=20) as executor:
        for name in stocks_lst:
            file_name = name
            threads.append(executor.submit(download_file, file_name))

runner()


In [123]:
us_stock_lst

[yfinance.Ticker object <ABT>,
 yfinance.Ticker object <AIG>,
 yfinance.Ticker object <AMZN>,
 yfinance.Ticker object <BAC>,
 yfinance.Ticker object <COF>,
 yfinance.Ticker object <CAT>,
 yfinance.Ticker object <BLK>,
 yfinance.Ticker object <ACN>,
 yfinance.Ticker object <ABBV>,
 yfinance.Ticker object <BA>,
 yfinance.Ticker object <AXP>,
 yfinance.Ticker object <CL>,
 yfinance.Ticker object <BK>,
 yfinance.Ticker object <COP>,
 yfinance.Ticker object <CMCSA>,
 yfinance.Ticker object <BMY>,
 yfinance.Ticker object <BIIB>,
 yfinance.Ticker object <C>,
 yfinance.Ticker object <CSCO>,
 yfinance.Ticker object <COST>,
 yfinance.Ticker object <JPM>,
 yfinance.Ticker object <KMI>,
 yfinance.Ticker object <MO>,
 yfinance.Ticker object <GOOG>,
 yfinance.Ticker object <GM>,
 yfinance.Ticker object <MSFT>,
 yfinance.Ticker object <MON>,
 yfinance.Ticker object <LLY>,
 yfinance.Ticker object <CVS>,
 yfinance.Ticker object <OXY>,
 yfinance.Ticker object <LMT>,
 yfinance.Ticker object <NKE>,
 yfina

In [125]:
len(us_stock_lst)

54

### Step 2: Filter out tickers within date range based on daily average volume

In [119]:
# Define a list to store the valid tickers
valid_stock_list = []

# time interval of July 02, 2021 to October 22, 2021
start_date = "2021-07-02"
end_date = "2021-10-22"

# Finds the average daily volume of a stock
def avg_daily_volume(ticker):
    ticker_history = ticker.history(start=start_date,end=end_date,interval='1d')
    ticker_vol_avg = ticker_history['Volume'].mean()
    return ticker_vol_avg
    
    
# iterates through us_stock_lst and checks if the stock has an average daily volume of at least 10,000 shares and appends to a list
def volume_filter(ticker_lst):
    for i in us_stock_lst:
        if avg_daily_volume(i) >= 10_000:
            valid_stock_list.append(i)
        else:
            continue
    return valid_stock_list

filtered_stocks = volume_filter(us_stock_lst)
filtered_stocks

KeyboardInterrupt: 

In [108]:
len(filtered_stocks)

54

### Step #3 Obtain the 20 most volatile stocks from the filtered lsit

In [109]:
def find_volatile (stock_count, ticker_lst, start_date, end_date): #Stock count will most likely be 20 #ticker_lst will always be filtered_stocks
    #Get the stock history
    stock_history = []
    for y in range (len (ticker_lst)):
        stock_history.append(ticker_lst[y].history(start=start_date, end=end_date))
    
    #Monthly Closing Prices dataframe
    monthly_close = []
    for g in range (len(stock_history)):
        monthly_close.append(pd.DataFrame(stock_history[g]['Close']))
    
    #Combine the individual stocks together to make one large dataframe
    share_prices = pd.concat (monthly_close, join='inner', axis=1)

    # Rename the columns (will need to fix this soon)
    share_prices.columns = ticker_lst

    #Convert our dataframe into simply one column with standard deviation 
    sd_of_shares = share_prices.pct_change().std()

    #Have to convert sd_of_shares into a dataframe
    sd_df = pd.DataFrame(sd_of_shares)

    #Find the 20 largest standard deviations meaning the most volatile
    column = sd_df[0]
    largest_names = []
    for g in range (stock_count):
        largest_names.append(column.idxmax().info['symbol'])
        column = column.drop([column.idxmax()])

    return largest_names

largest_volatile = find_volatile(20, filtered_stocks, '2021-07-02', '2021-10-22')    

In [110]:
largest_volatile

['OXY',
 'SLB',
 'GM',
 'COP',
 'COF',
 'BA',
 'PYPL',
 'AIG',
 'SPG',
 'BIIB',
 'BAC',
 'USB',
 'KMI',
 'BK',
 'AXP',
 'CAT',
 'MS',
 'PFE',
 'AMZN',
 'CMCSA']

In [70]:
#How to get the ticker of a specific stock
largest_volatile.index[5].info['symbol']

'AXP'

### Step #4: Find the porfolio with the best correlation

In [1]:
# Define a global price dataframe to store all past data
global_price_df = pd.DataFrame()

NameError: name 'pd' is not defined

In [14]:
### Add all data for tickers to a global price dataframe
def update_price_df(ticker_lst, start_date, end_date):
    # Load the global dataframe
    global global_price_df
    
    # Read in the history closing price info
    # Store in a dictionary where the key is the name of the ticker
    hist_dic = {}
    for i in ticker_lst:
        if i not in global_price_df:
            ticker_yf = yf.Ticker(i)
            hist_dic[i] = ticker_yf.history(start=start_date, end=end_date)['Close']
    
    # Convert the dictionary to a dataframe
    temp_price_df = pd.DataFrame(hist_dic)
    
    # If there is tickers to add, then resample it to month and add to the global closing price df
    if not temp_price_df.empty:
        # temp_prices_df = prices_df.resample('MS').first()
        global_price_df = pd.concat([global_price_df, temp_price_df], axis=1)

In [15]:
### Find all possible combinations of n tickers within the ticker_lst using a recursive function
def combinations(arr, length, pre_arr=[]):
    if len(pre_arr) == length:
        return [pre_arr]
    
    combs = []
    # Add each element to the previous array
    for i, val in enumerate(arr):
        cur_copy = pre_arr.copy()
        cur_copy.append(val)
        combs += combinations(arr[i+1:], length, cur_copy)
    return combs

In [16]:
### Find the average correlation between each stock
def find_avg_corr(ticker_lst, start_date, end_date):
    # Load the global price dataframe
    global global_price_df
    
    # Find the dataframe with the given tickers and filter the dates
    price_df = global_price_df[ticker_lst]
    price_df = price_df[(start_date <= price_df.index) & (price_df.index <= end_date)]
    
    # Find the correlation matrix
    df_corr = price_df.corr()
    
    # Calculate the avg corr
    sum_corr = 0
    sum_count = 0
    for i in range (len(ticker_lst)):
        for j in range (i+1, len(ticker_lst)):
            sum_corr += df_corr.loc[ticker_lst[i],ticker_lst[j]]
            sum_count += 1
    
    # Calculate the return the avg correlation
    return sum_corr/sum_count

In [17]:
# Find the combinaton of tickers with the maximum avg correlation
def find_max_correlation(ticker_lst,  start_date, end_date, result_size = 10):
    # Store the maximum value of corr and the portfolio
    max_avg_corr = -1
    max_corr_port = np.array([])
    
    # Load the global price dataframe
    global global_price_df
    
    # Convert list to array and find the combinations
    comb = combinations(np.array(ticker_lst), result_size)
    
    # Find the comb with the maximum average correlation
    for i in comb:
        avg_corr = find_avg_corr(i, start_date, end_date)
        if avg_corr > max_avg_corr:
            max_avg_corr = avg_corr
            max_corr_port = i
    
    # Return the result sample
    return max_corr_port

In [18]:
### Applying the functions
# A random ticker list
ticker_lst = ['MSFT', 'AAPL', 'GOOGL', 'AMZN', 'TSLA', 'FB', 'NVDA', 'NFLX', 'TSM', 'JPM',
             'BABA', 'V', 'JNJ', 'UNH', 'WMT'] #, 'HD', 'BAC', 'MA', 'ASML', 'PG']

# Update the global closing df
update_price_df(ticker_lst, start_date, end_date)

# Find the combination of tickers with maximum correlation
max_corr_port = find_max_correlation(ticker_lst, start_date, end_date)

# Display
print('The portfolio with maximum avg correlation is:', max_corr_port)

The portfolio with maximum avg correlation is: ['MSFT', 'AAPL', 'GOOGL', 'TSLA', 'FB', 'NVDA', 'NFLX', 'JPM', 'UNH', 'WMT']


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here. 